# Topics in Muography 
#### Roland Grinis  -  Researcher at MIPT Nuclear Physics Methods lab  -  CTO at GrinisRIT (grinisrit.com)

Code available within NOA github.com/grinisrit/noa - Bayesian computation algorithms in C++17 over LibTorch

## Installation

The `conda` environment provided with the repository has all the required dependencies. For this particular tutorial we will need the following `python` packages:

In [1]:
import torch
from torch.utils.cpp_extension import load
import matplotlib.pyplot as plt
%matplotlib inline

If you prefer dark plots you can run:

In [2]:
import seaborn as sns
sns.set_style('darkgrid',
                {'axes.facecolor': '.2',
                'figure.facecolor': '0.1',
                'text.color': '.9',
                'grid.color': '.5',
                'xtick.color': '.9',
                'ytick.color': '.9'})

Now we need to build and load `C++17/CUDA` extensions for `PyTorch`, set up the locations:

In [4]:
!mkdir -p build
noa_location = '../..'

If you are running this on Google Colab, you need to clone `NOA` and set `noa_location` accordingly:
```python
!git clone https://github.com/grinisrit/noa.git
noa_location = 'noa'
```

Also, make sure that `ninja` and `g++-8` or higher are available. The following commands will do that for you:
```python
!pip install Ninja
!apt install gcc-8 g++-8 -y
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 1000
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-8 1000
!gcc --version
!g++ --version
!nvcc --version
```
Finally, you get the extensions into `python` by calling `load`:

In [5]:
muons = load(name='muons',
             build_directory='./build',
             sources=[f'{noa_location}/docs/pms/muons.cc'],
             extra_include_paths=[f'{noa_location}/include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             extra_ldflags=['-lstdc++fs'],
             verbose=False)

In [6]:
muons_cuda = load(name='muons_cuda',
             build_directory='./build',       
             sources=[f'{noa_location}/docs/pms/muons.cu'],
             extra_include_paths=[f'{noa_location}/include'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             extra_cuda_cflags=['-std=c++17 --extended-lambda'],
             extra_ldflags=['-lstdc++fs'],
             verbose=False) if torch.cuda.is_available() else None

## Differential Cross-Sections calculations

In [7]:
kinetic_energies = torch.linspace(1e-3, 1e6, 10000).double()
recoil_energies = 0.0505 * kinetic_energies

In [8]:
kinetic_energies_gpu = kinetic_energies.cuda()
recoil_energies_gpu = recoil_energies.cuda()

In [9]:
kinetic_energies_gpu[:5]

tensor([1.0000e-03, 1.0001e+02, 2.0002e+02, 3.0003e+02, 4.0004e+02],
       device='cuda:0', dtype=torch.float64)

In [11]:
brems = muons.bremsstrahlung(kinetic_energies, recoil_energies)
brems[:5]

tensor([3.5293e-04, 3.9395e-06, 4.0777e-06, 4.1341e-06, 4.1650e-06],
       dtype=torch.float64)

In [12]:
brems_gpu = muons_cuda.bremsstrahlung(kinetic_energies_gpu, recoil_energies_gpu);
brems_gpu[:5]

tensor([3.5293e-04, 3.9395e-06, 4.0777e-06, 4.1341e-06, 4.1650e-06],
       device='cuda:0', dtype=torch.float64)

In [16]:
(brems - brems_gpu.cpu()).abs().sum()

tensor(3.2179e-18, dtype=torch.float64)

In [14]:
%timeit muons_cuda.bremsstrahlung(kinetic_energies_gpu, recoil_energies_gpu);

151 µs ± 5.64 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%timeit muons.bremsstrahlung(kinetic_energies, recoil_energies);

285 µs ± 3.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
